In [64]:
import random
from deap import base, creator, tools, algorithms
import pandas as pd
import numpy as np
from typing import Tuple
import xlsxwriter
# Dear Kuba if you don't have some module you can install it with %pip install xlsxwriter and run the cell.

In [87]:
MAX_EVAL: int = 10000
MAX_ALGO: int = 100
DOMAIN_R = (-30, 30)
DIMENSIONS: Tuple = (5, 15, 30)

### Simplified Rosenbrock Function (Generalized)
The Rosenbrock function (generalized to higher dimensions) is defined as:

$$
f(x) = \sum_{j=0}^{D-2} \left[100(x_{j+1} - x_j^2)^2 + (x_j - 1)^2\right]
$$

In [66]:
def rosenbrock(individual):
    # Decode binary representation to real values in [-30, 30]
    dim = len(individual) // 16  # Number of dimensions
    values = [
        int("".join(map(str, individual[i * 16:(i + 1) * 16])), 2) / (2**16 - 1) * 60 - 30
        for i in range(dim)
    ]
    # Calculate Rosenbrock value
    return sum((1 - values[i])**2 + 100 * (values[i + 1] - values[i]**2)**2 for i in range(dim - 1)),

# ^
Chatbot's function

My function
# v

In [67]:
def my_rosenbrock(individual):
    """
    Calculate the Rosenbrock function value for an individual.
    Decodes binary representation to real values within DOMAIN_R.
    
    Args:
        individual: List of binary values representing the individual.

    Returns:
        A tuple containing the Rosenbrock function value for the decoded real values.
    """
    dim = len(individual) // 16  # Number of dimensions
    lower, upper = DOMAIN_R  # Extract domain bounds
    range_width = upper - lower

    # Decode binary representation to real values in DOMAIN_R
    values = [
        int("".join(map(str, individual[i * 16:(i + 1) * 16])), 2) / (2**16 - 1) * range_width + lower
        for i in range(dim)
    ]

    # Calculate the Rosenbrock function value
    result = sum(
        100 * (values[i + 1] - values[i]**2)**2 + (values[i] - 1)**2
        for i in range(dim - 1)
    )

    # Return the result as a tuple
    return result,

# Rosenbrock's setup
- Binary repr (16 bits per dimention)
- Looking for minimum
- Selection: Tournament
- Crossover: Two-point crossover
- Mutation: Flip-bit
- Succession: Elitism


# TODO Salomon's setup

- Selection: SUS (Stochastic Universal Sampling) for better exploration.
- Crossover: Uniform crossover for diverse offspring.
- Mutation: Gaussian mutation for finer-grain adjustments.
- Succession: Elitism to preserve top performers.

# TODO Whitley's

- Selection: Rank selection to handle deceptive fitness peaks.
- Crossover: Two-point crossover for local exploitation.
- Mutation: Flip-bit mutation for exploration.
- Succession: Steady-state evolution to maintain diversity.

In [68]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Global minimization
creator.create("Individual", list, fitness=creator.FitnessMin)

In [69]:
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)  # Binary gene (0 or 1)
toolbox.register("evaluate", rosenbrock)
toolbox.register("mate", tools.cxTwoPoint)  # Two-point crossover
toolbox.register("mutate", tools.mutFlipBit, indpb=0.01)  # Flip bit mutation
toolbox.register("select", tools.selTournament, tournsize=3) 

# This is test algo

def run_algorithm():
    population = toolbox.population(n=300)
    evaluations = 0

    run_data = []  # To store real values for this run

    while evaluations < MAX_EVAL:
        offspring = algorithms.varAnd(population, toolbox, cxpb=0.7, mutpb=0.2)
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]

        # Evaluate fitness for invalid individuals
        for ind in invalid_ind:
            ind.fitness.values = toolbox.evaluate(ind)
            evaluations += 1

            # Decode binary to real values and store them
            real_values = [
                int("".join(map(str, ind[i * 16:(i + 1) * 16])), 2) / (2**16 - 1) * 60 - 30
                for i in range(5)
            ]
            run_data.extend(real_values)

            # Stop if max evaluations are reached
            if evaluations >= MAX_EVAL:
                break

        # Replace population with new generation
        population[:] = toolbox.select(offspring + population, k=len(population))

    return run_data

### Run the GA 100 times and collect data
raw_data = {f"Run_{i + 1}": [] for i in range(MAX_ALGO)}

for i in range(MAX_ALGO):
    print(f"Running GA for column {i + 1}...")
    raw_data[f"Run_{i + 1}"] = run_algorithm()

### Save results to Excel
df = pd.DataFrame(raw_data)
df.to_excel("rosenbrock_results_fixed.xlsx", index=False)

# This is actual algo

In [88]:
def run_algorithm_2(dimension, pop_size, cxpb, mutpb):
    # Register individual and population specific to the dimension
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=16 * dimension)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Initialize population
    population = toolbox.population(n=pop_size)
    evaluations = 0
    fitness_data = []  # Store fitness values for ECDF plotting
    local_max_eval: int = MAX_EVAL * dimension

    while evaluations < local_max_eval:
        # Generate offspring
        offspring = algorithms.varAnd(population, toolbox, cxpb=cxpb, mutpb=mutpb)
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]

        # Evaluate invalid individuals
        for ind in invalid_ind:
            ind.fitness.values = toolbox.evaluate(ind)  # Evaluate fitness
            evaluations += 1

            # Append the fitness value to fitness_data
            fitness_data.append(ind.fitness.values[0])

            # Stop if the evaluation budget is exhausted
            if evaluations >= local_max_eval:
                break

        # Replace population with the new generation
        population[:] = toolbox.select(offspring + population, k=len(population))

    return fitness_data


In [89]:
writer = pd.ExcelWriter("rosenbrock_results.xlsx", engine='xlsxwriter')

for dim in DIMENSIONS:
    print(f"=== Starting GA for dimensions: {dim} ===")
    pop_size = 200 if dim == 5 else (300 if dim == 15 else 500)
    cxpb = 0.8
    mutpb = 0.02 if dim == 5 else (0.03 if dim == 15 else 0.05)

    # Prepare raw data dictionary for storing runs
    raw_data = {f"Run_{i + 1}": [] for i in range(MAX_ALGO)}

    for i in range(MAX_ALGO):
        print(f"  Running experiment {i + 1} for dimension {dim}...")
        raw_data[f"Run_{i + 1}"] = run_algorithm_2(dim, pop_size, cxpb, mutpb)

    # Convert to DataFrame and write to corresponding sheet
    df = pd.DataFrame(raw_data)
    df.to_excel(writer, sheet_name=f"rosenbrock_{dim}", index=False)

writer.close()
print("Results saved to rosenbrock_results.xlsx")


=== Starting GA for dimensions: 5 ===
  Running experiment 1 for dimension 5...
  Running experiment 2 for dimension 5...
  Running experiment 3 for dimension 5...
  Running experiment 4 for dimension 5...
  Running experiment 5 for dimension 5...
  Running experiment 6 for dimension 5...
  Running experiment 7 for dimension 5...
  Running experiment 8 for dimension 5...
  Running experiment 9 for dimension 5...
  Running experiment 10 for dimension 5...
  Running experiment 11 for dimension 5...
  Running experiment 12 for dimension 5...
  Running experiment 13 for dimension 5...
  Running experiment 14 for dimension 5...
  Running experiment 15 for dimension 5...
  Running experiment 16 for dimension 5...
  Running experiment 17 for dimension 5...
  Running experiment 18 for dimension 5...
  Running experiment 19 for dimension 5...
  Running experiment 20 for dimension 5...
  Running experiment 21 for dimension 5...
  Running experiment 22 for dimension 5...
  Running experiment 23 f